Create the model (should be able to read from file, but doesnt)

In [1]:
from Declare4Py.ProcessModels.DeclareModel import DeclareModel, DeclareModelTemplate
from Declare4Py.D4PyEventLog import D4PyEventLog
from Declare4Py.ProcessMiningTasks.Discovery.DeclareMiner import DeclareMiner 
import os

In [2]:
log_path = os.path.join("../data/extension_log/extension-log-4.xes")
event_log = D4PyEventLog(case_name="case:concept:name")
event_log.parse_xes_log(log_path)
extension_log_model = DeclareMiner(log=event_log, consider_vacuity=False, min_support=0.2, itemsets_support=0.9, max_declare_cardinality=3).run()

/home/theo/DTU/02269 Process mining/declare4py_test/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 1000/1000 [00:00<00:00, 2424.17it/s]


Computing discovery ...


In [3]:
extension_log_model

Conformance checking with noisy log

In [4]:
log_path = os.path.join("../data/extension_log/extension-log-noisy-4.xes")
event_log = D4PyEventLog(case_name="case:concept:name")
event_log.parse_xes_log(log_path)

parsing log, completed traces :: 100%|██████████| 1000/1000 [00:00<00:00, 3119.64it/s]


In [5]:
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareResultsBrowser import MPDeclareResultsBrowser

conf_check_res = MPDeclareAnalyzer(log=event_log, declare_model=extension_log_model, consider_vacuity=False).run()


In [6]:
state_df = conf_check_res.get_metric(metric="state")

In [14]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from pybeamline.sources import log_source
from pybeamline.mappers import sliding_window_to_log
from reactivex.operators import window_with_count, take
import pm4py

logs = []
log_source("../data/extension_log/extension-log-noisy-4.xes").pipe(
    window_with_count(100),
    sliding_window_to_log(),
    take(1)
).subscribe(lambda x: logs.append(pm4py.convert_to_event_log(x)))

parsing log, completed traces :: 100%|██████████| 1000/1000 [00:00<00:00, 6148.17it/s]
/home/theo/DTU/02269 Process mining/declare4py_test/env/lib/python3.12/site-packages/reactivex/internal/basic.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow()
/home/theo/DTU/02269 Process mining/declare4py_test/env/lib/python3.12/site-packages/reactivex/internal/basic.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow()
/home/theo/DTU/02269 Process mining/declare4py_test/env/lib/python3.12/site-packages/reactivex/internal/basic.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future ver

In [19]:
for l in logs:
    for m in l:
        print(m)

{'attributes': {'concept:name': 'case_868'}, 'events': [{'concept:name': 'record issue', 'time:timestamp': Timestamp('1970-01-01 01:00:00+0000', tz='UTC'), 'org:resource': 'admin-2', 'cost': 14, 'process:concept:name': 'log-file'}]}
{'attributes': {'concept:name': 'case_923'}, 'events': [{'concept:name': 'record issue', 'time:timestamp': Timestamp('1970-01-01 01:00:00+0000', tz='UTC'), 'org:resource': 'admin-2', 'cost': 14, 'process:concept:name': 'log-file'}]}
{'attributes': {'concept:name': 'case_750'}, 'events': [{'concept:name': 'record issue', 'time:timestamp': Timestamp('1970-01-01 01:00:00+0000', tz='UTC'), 'org:resource': 'admin-3', 'cost': 14, 'process:concept:name': 'log-file'}]}
{'attributes': {'concept:name': 'case_499'}, 'events': [{'concept:name': 'record issue', 'time:timestamp': Timestamp('1970-01-01 01:00:00+0000', tz='UTC'), 'org:resource': 'admin-1', 'cost': 12, 'process:concept:name': 'log-file'}]}
{'attributes': {'concept:name': 'case_402'}, 'events': [{'concept:na

In [10]:
for l in logs:
    l._properties['pm4py:param:timestamp_key'] = 'time:timestamp'
    l._properties['pm4py:param:activity_key'] = 'concept:name'

test = D4PyEventLog(case_name="case:concept:name", log=logs[0])


In [11]:
event_log.log._properties

{'pm4py:param:activity_key': 'concept:name',
 'pm4py:param:attribute_key': 'concept:name',
 'pm4py:param:timestamp_key': 'time:timestamp',
 'pm4py:param:resource_key': 'org:resource',
 'pm4py:param:transition_key': 'lifecycle:transition',
 'pm4py:param:group_key': 'org:group'}

In [12]:
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareResultsBrowser import MPDeclareResultsBrowser

basic_checker = MPDeclareAnalyzer(log=test, declare_model=extension_log_model, consider_vacuity=False)
conf_check_res: MPDeclareResultsBrowser = basic_checker.run()

In [13]:
conf_check_res.get_metric(metric="state")

,Existence1[record issue] | |,Absence2[record issue] | |,Absence3[record issue] | |,Exactly1[record issue] | |,Init[record issue] | |,Existence1[issue completion] | |,Absence2[issue completion] | |,Absence3[issue completion] | |,Exactly1[issue completion] | |,End[issue completion] | |,...,"Response[inspection, issue completion] | |","Alternate Response[inspection, issue completion] | |","Precedence[inspection, issue completion] | |","Alternate Precedence[inspection, issue completion] | |","Not Response[issue completion, inspection] | |","Not Precedence[issue completion, inspection] | |","Not Chain Response[inspection, issue completion] | |","Not Chain Response[issue completion, inspection] | |","Not Chain Precedence[inspection, issue completion] | |","Not Chain Precedence[issue completion, inspection] | |"
0,1,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
